In [4]:
import json
from pathlib import Path

with open('correct_files.json','r+') as jfile:
    selected_files = json.load(jfile)

files_to_load = [Path(x) for x in selected_files]

In [7]:
import pandas as pd
from tqdm import tqdm

df_list = list()
for file in tqdm(files_to_load):
    df_list.append(pd.read_parquet(file))
gdf = pd.concat(df_list)

100%|██████████| 361/361 [00:22<00:00, 15.95it/s]


In [22]:
gdf['Start Date'] = pd.to_datetime(gdf['Start Date'],exact=False)
gdf['End Date'] = pd.to_datetime(gdf['End Date'],exact=False)

In [27]:
gdf.count()

Rental Id            80773711
Duration             80607888
Bike Id              80773693
End Date             80607888
EndStation Id        80606434
EndStation Name      80606434
Start Date           80773711
StartStation Id      80773711
StartStation Name    80773711
dtype: int64

In [32]:
from pandas_profiling import ProfileReport
profile = ProfileReport(gdf, minimal=True)
profile.to_file("Big_Dataset_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [28]:
gdf['Bike Id'] = gdf['Bike Id'].astype(pd.Int64Dtype())
gdf['Duration'] = gdf['Duration'].astype(pd.Int64Dtype())
gdf['EndStation Id'] = gdf['EndStation Id'].astype(pd.Int64Dtype())

In [30]:
gdf.to_parquet('full_data_uniform.parquet')

In [11]:
from dask.diagnostics import progress
import dask.dataframe as ddf

with progress.ProgressBar():
    dgdf = ddf.from_pandas(gdf,32)

In [ ]:
dgdf['End Date'] = ddf.to_datetime(dgdf['End Date'],exact=False,errors='coerce',infer_datetime_format=True)
dgdf['Start Date'] = ddf.to_datetime(dgdf['Start Date'],exact=False,errors='coerce',infer_datetime_format=True)
with progress.ProgressBar():
    dgdf.compute()